In [ ]:
# 模型构造
# 构造Sequential实例，然后依次添加两个全连接层。其中一层的输出大小为256，即隐藏层单元个数是256；
# 第二层输出大小为10，即输出单元个数是10

In [1]:
import torch
from torch import nn


In [4]:
class MLP(nn.Module):
  def __init__(self, **kwargs):
    # 调用MLP父类Module的构造函数来进行必要的初始化。
    # 参数，params
    super(MLP,self).__init__(**kwargs)
    self.hidden = nn.Linear(784,256) # 隐藏层
    self.act = nn.ReLU()
    self.output = nn.Linear(256,10) # 输出层

  def forward(self,x):
    # 定义模型的前向计算，即如何根据输入x计算返回所需要的模型输出
    a = self.act(self.hidden(x))
    return self.output(a)

'''
  以上的MLP类中无序定义反向传播函数,系统将通过自动求梯度而自动生成反向传播所需的backward函数
'''

'\n  以上的MLP类中无序定义反向传播函数,系统将通过自动求梯度而自动生成反向传播所需的backward函数\n'

In [5]:
X = torch.rand(2,784)
net = MLP()
print(net)
net(X)


MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.2343, -0.1191,  0.0747, -0.1021, -0.0153,  0.1037, -0.1777, -0.0339,
         -0.2016,  0.2286],
        [ 0.1334, -0.1232,  0.1454, -0.0680, -0.0735,  0.0268, -0.1940, -0.0291,
         -0.1422,  0.0546]], grad_fn=<AddmmBackward0>)

In [8]:
from typing import OrderedDict


class MySequential(nn.Module):
  from collections import OrderedDict

  def __init__(self, *args):
    super(MySequential, self).__init__()
    if len(args) == 1 and isinstance(args[0], OrderedDict):
      for key, module in args[0].items():
        self.add_module(key, module)
    else:
      for idx, module in enumerate(args):
        self.add_module(str(idx), module)

  def forward(self, input):
    for module in self._modules.values():
      input = module(input)
    return input


In [13]:
net = MySequential(
  nn.Linear(784, 256),
  nn.ReLU(),
  nn.Linear(256, 10),
)
print(net)
net(X)


MySequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-0.0339,  0.0454,  0.1566,  0.1904,  0.1290, -0.0395,  0.0175,  0.1380,
          0.1985, -0.2610],
        [-0.0224,  0.0787,  0.0346,  0.0021,  0.3048,  0.1256, -0.1011,  0.0629,
          0.2077, -0.1226]], grad_fn=<AddmmBackward0>)

In [14]:
net = nn.ModuleList([nn.Linear(784,256),nn.ReLU()])
net.append(nn.Linear(256,10))
print(net[-1])
print(net)

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


In [16]:
class MyModule(nn.Module):
  def __init__(self):
    super(MyModule,self).__init__()
    self.linears = nn.ModuleList([nn.Linear(10,10) for i in range(10)])
  
  def forward(self,x):
    for i, l in enumerate(self.linears):
      x = self.linears[i // 2](x)+l(x)
    return x

In [18]:
net = MyModule()
print(net)

MyModule(
  (linears): ModuleList(
    (0-9): 10 x Linear(in_features=10, out_features=10, bias=True)
  )
)


In [19]:
net = nn.ModuleDict({
  "linear":nn.Linear(784,256),
  "act":nn.ReLU(),
})

net["output"] = nn.Linear(256,10)
print(net["linear"])
print(net.output)
print(net)

Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


In [25]:
# 构造复杂的模型
class FancyMLP(nn.Module):
  def __init__(self, **kwargs):
    super(FancyMLP, self).__init__(**kwargs)
    self.rand_weight = torch.rand((20, 20), requires_grad=False)
    self.linear = nn.Linear(20, 20)

  def forward(self, x):
    x = self.linear(x)
    # 使用创建的常数参数，以及nn.functional 中的relu函数和mm函数
    x = nn.functional.relu(torch.mm(x, self.rand_weight.data) + 1)
    # 复用全连接层
    x = self.linear(x)
    # 控制流，这里我们需要调用item函数来返回标量进行比较
    while x.norm().item() > 1:
      x /= 2

    if x.norm().item() < 0.8:
      x *= 10

    return x.sum()


In [26]:
X = torch.rand(2,20)
net = FancyMLP()
print(net)
net(X)

FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)


tensor(0.1856, grad_fn=<SumBackward0>)

In [27]:
class NestMLP(nn.Module):
  def __init__(self, **kwargs):
    super(NestMLP, self).__init__(**kwargs)
    self.net = nn.Sequential(nn.Linear(40, 30), nn.ReLU())

  def forward(self, x):
    return self.net(x)


In [28]:
# FancyMLP和Sequential都是Module的子类，我们可以嵌套调用
net = nn.Sequential(NestMLP(),nn.Linear(30,20),FancyMLP())
X = torch.rand(2,40)
print(net)
net(X)

Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=40, out_features=30, bias=True)
      (1): ReLU()
    )
  )
  (1): Linear(in_features=30, out_features=20, bias=True)
  (2): FancyMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)


tensor(9.1595, grad_fn=<SumBackward0>)